# Lab 06 - Programación Lineal
#### **Modelación y Simulación Sección 10**

~ Samuel Chamale 21885

~ Adrian Rodriguez 21691

~ Daniel Gomez 21429

# 1. (Modelo de Producción, Período único)

In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")

In [2]:
using JuMP, HiGHS

In [3]:
function solve_model(is_integer::Bool)
    # Define the model
    model = Model()

    # Decision variables
    if is_integer
        @variable(model, x1 >= 0, Int)  # Chamarras
        @variable(model, x2 >= 0, Int)  # Relleno de plumas
        @variable(model, x3 >= 0, Int)  # Pantalones
        @variable(model, x4 >= 0, Int)  # Guantes
    else
        @variable(model, x1 >= 0)  # Chamarras
        @variable(model, x2 >= 0)  # Relleno de plumas
        @variable(model, x3 >= 0)  # Pantalones
        @variable(model, x4 >= 0)  # Guantes
    end

    # Variables de holgura para la demanda no satisfecha
    @variable(model, s1 >= 0)  # Chamarras
    @variable(model, s2 >= 0)  # Relleno de plumas
    @variable(model, s3 >= 0)  # Pantalones
    @variable(model, s4 >= 0)  # Guantes

    # Variables de holgura para la capacidad no utilizada
    @variable(model, s5 >= 0)  # Corte
    @variable(model, s6 >= 0)  # Aislamiento
    @variable(model, s7 >= 0)  # Costura
    @variable(model, s8 >= 0)  # Empaque

    # Funcion objetivo
    @objective(model, Max,
        30x1 + 40x2 + 20x3 + 10x4 - 15s1 - 20s2 - 10s3 - 8s4
    )

    # Restricciones de demanda
    @constraint(model, x1 + s1 == 800)
    @constraint(model, x2 + s2 == 750)
    @constraint(model, x3 + s3 == 600)
    @constraint(model, x4 + s4 == 500)

    # Restricciones de capacidad
    @constraint(model, 0.30x1 + 0.30x2 + 0.25x3 + 0.15x4 + s5 == 1000)  # Corte
    @constraint(model, 0.25x1 + 0.35x2 + 0.30x3 + 0.10x4 + s6 == 1000)  # Aislamiento
    @constraint(model, 0.45x1 + 0.50x2 + 0.40x3 + 0.22x4 + s7 == 1000)  # Costura
    @constraint(model, 0.15x1 + 0.15x2 + 0.10x3 + 0.05x4 + s8 == 1000)  # Empaque

    set_optimizer(model, HiGHS.Optimizer)

    optimize!(model)

    println(is_integer ? "\nCon restricciones de enteros:" : "\nSin restricciones de enteros:")
    println("Chamarras: ", value(x1))
    println("Relleno de plumas: ", value(x2))
    println("Pantalones: ", value(x3))
    println("Guantes: ", value(x4))
    println("\nFaltantes en la producción:")
    println("Chamarras faltantes: ", value(s1))
    println("Relleno de plumas faltantes: ", value(s2))
    println("Pantalones faltantes: ", value(s3))
    println("Guantes faltantes: ", value(s4))
    println("\nCapacidad no utilizada (horas):")
    println("Corte: ", value(s5))
    println("Aislamiento: ", value(s6))
    println("Costura: ", value(s7))
    println("Empaque: ", value(s8))
    println("\nUtilidad Total: \$", objective_value(model))
end

solve_model (generic function with 1 method)

In [4]:
solve_model(false)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [8e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 1e+03]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
Presolve : Reductions: rows 1(-7); columns 5(-7); elements 5(-23)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -6.4625000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  6.4625000000e+04
HiGHS run time      :          0.01

Sin restricciones de enteros:
Chamarras: 800.0
Relleno de plumas: 750.0
Pantalones: 387.5
Guantes: 500.0

Faltantes en la producción:
Chamarras faltantes: 0.0
Relleno de plumas faltantes: 0.0
Pantalones faltantes: 212.5
Guantes faltantes: 0.0

C

### Versión con restricciones enteras

In [5]:
solve_model(true)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-02, 1e+00]
  Cost   [8e+00, 4e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 1e+03]
Presolving model
1 rows, 5 cols, 5 nonzeros  0s
1 rows, 5 cols, 5 nonzeros  0s
Objective function is integral with scale 0.333333

Solving MIP model with:
   1 rows
   5 cols (0 binary, 4 integer, 1 implied int., 0 continuous)
   5 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   71000           -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   64625           64595              0.05%        0      0      0         1     0.0s
 C       0       0         0   0.00%   64622           64610 

# 2. (Modelo de Producción, Períodos múltiples)

In [6]:
import Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("DataFrames")

    Updating registry at `C:\Users\chama\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\chama\.julia\environments\v1.10\Manifest.toml`


In [7]:
using JuMP
using HiGHS
using DataFrames

In [8]:
months = 1:6
demand = [180, 250, 190, 140, 220, 250]
production_cost = [50, 45, 55, 52, 48, 50]
inventory_cost = [8, 10, 10, 10, 8, 8]
capacity = 225


model = Model(HiGHS.Optimizer)

@variable(model, x[months] >= 0, start=0)  # x[m] = la cantidad de ventanas producidas en el mes m
@variable(model, I[months] >= 0, start=0)  # I[m] = inventario al final del mes m

# Capacidad de producción
@constraint(model, [m in months], x[m] <= capacity)

# Balance de inventario
@constraint(model, I[1] == x[1] - demand[1])
@constraint(model, [m in 2:6], I[m] == I[m-1] + x[m] - demand[m])

# Minimizar el costo total, que es la suma de los costos de producción y de inventario
@objective(model, Min,
    sum(production_cost[m] * x[m] + inventory_cost[m] * I[m] for m in months)
)


optimize!(model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(1230) 0s
         13     6.1795000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 13
Objective value     :  6.1795000000e+04
HiGHS run time      :          0.00


In [9]:
status = termination_status(model)

if status == MOI.OPTIMAL
    println("Optimal solution found.\n")

    production_values = collect(value.(x))
    inventory_values = collect(value.(I))

    direct_production_cost = [demand[i] * production_cost[i] for i in months] # asumiendo que se puede producir exactamente la demanda
    optimal_production_cost = [production_values[i] * production_cost[i] for i in months]

    results = DataFrame(
        :Month => months,
        :Production => production_values,
        :Inventory => inventory_values,
        :Optimal_Production_Cost =>  optimal_production_cost,
        :Direct_Production_Cost => direct_production_cost,
        :Difference =>   optimal_production_cost .- direct_production_cost
    )

    println(results)
    println("\nTotal cost: \$", objective_value(model))
    println("Naive production cost: \$", sum(direct_production_cost))
    println("Cost difference: \$", sum(results.Difference))

else
    println("Optimization was not successful. Status: ", status)
end


Optimal solution found.

6×6 DataFrame
 Row │ Month  Production  Inventory  Optimal_Production_Cost  Direct_Production_Cost  Difference 
     │ Int64  Float64     Float64    Float64                  Int64                   Float64    
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │     1       205.0       25.0                  10250.0                    9000      1250.0
   2 │     2       225.0        0.0                  10125.0                   11250     -1125.0
   3 │     3       190.0        0.0                  10450.0                   10450         0.0
   4 │     4       160.0       20.0                   8320.0                    7280      1040.0
   5 │     5       225.0       25.0                  10800.0                   10560       240.0
   6 │     6       225.0        0.0                  11250.0                   12500     -1250.0

Total cost: $61795.0
Naive production cost: $61040
Cost difference: $155.0


### Versión con restricciones enteras

In [10]:
model_integer = Model(HiGHS.Optimizer)

@variable(model_integer, x[months] >= 0, Int)  # Producción entera en cada mes
@variable(model_integer, I[months] >= 0, Int)  # Inventario entero al final de cada mes

@constraint(model_integer, [m in months], x[m] <= capacity)

@constraint(model_integer, I[1] == x[1] - demand[1])
@constraint(model_integer, [m in 2:6], I[m] == I[m-1] + x[m] - demand[m])

@objective(model_integer, Min,
    sum(production_cost[m] * x[m] + inventory_cost[m] * I[m] for m in months)
)

optimize!(model_integer)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [8e+00, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+02]
Presolving model
5 rows, 11 cols, 15 nonzeros  0s
3 rows, 9 cols, 11 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      61795
  Dual bound        61795
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    61795 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [11]:
status_integer = termination_status(model_integer)

if status_integer == MOI.OPTIMAL
    println("\nSolución óptima encontrada (Variables Enteras).\n")

    production_values_int = [value(x[m]) for m in months]
    inventory_values_int = [value(I[m]) for m in months]

    direct_production_cost_int = demand .* production_cost

    results_integer = DataFrame(
        Month = collect(months),
        Production = production_values_int,
        Inventory = inventory_values_int,
        Optimal_Production_Cost = production_values_int .* production_cost,
        Direct_Production_Cost = direct_production_cost_int
    )

    println(results_integer)
    println("\nTotal cost: \$", objective_value(model))
    println("Naive production cost: \$", sum(direct_production_cost))
    println("Cost difference: \$", sum(results.Difference))

else
    println("La optimización no fue exitosa (Entero). Estado: ", status_integer)
end


Solución óptima encontrada (Variables Enteras).

6×5 DataFrame
 Row │ Month  Production  Inventory  Optimal_Production_Cost  Direct_Production_Cost 
     │ Int64  Float64     Float64    Float64                  Int64                  
─────┼───────────────────────────────────────────────────────────────────────────────
   1 │     1       205.0       25.0                  10250.0                    9000
   2 │     2       225.0        0.0                  10125.0                   11250
   3 │     3       190.0        0.0                  10450.0                   10450
   4 │     4       160.0       20.0                   8320.0                    7280
   5 │     5       225.0       25.0                  10800.0                   10560
   6 │     6       225.0        0.0                  11250.0                   12500

Total cost: $61795.0
Naive production cost: $61040
Cost difference: $155.0

6×5 DataFrame
 Row │ Month  Production  Inventory  Optimal_Production_Cost  Direct_Producti

# 3. (Modelo de asignación de horarios)

In [13]:
import Pkg; Pkg.add("GLPK")

   Resolving package versions...
   Installed GLPK_jll ─ v5.0.1+0
   Installed GLPK ───── v1.2.1
    Updating `C:\Users\chama\.julia\environments\v1.10\Project.toml`
  [60bf3e95] + GLPK v1.2.1
    Updating `C:\Users\chama\.julia\environments\v1.10\Manifest.toml`
  [60bf3e95] + GLPK v1.2.1
  [e8aa6df9] + GLPK_jll v5.0.1+0
  [781609d7] + GMP_jll v6.2.1+6
Precompiling project...
  ✓ GLPK_jll
  ✓ GLPK
  2 dependencies successfully precompiled in 8 seconds. 280 already precompiled.


In [16]:
using JuMP, GLPK

# Create a model
model = Model(GLPK.Optimizer)

# Define variables
@variable(model, x[1:6] >= 0)

# Objective function: Minimize the total number of buses
@objective(model, Min, sum(x[i] for i in 1:6))

# Constraints
@constraint(model, x[1] + x[6] >= 4)    # 12:00 AM - 4:00 AM
@constraint(model, x[1] + x[2] >= 8) # 4:00 AM - 8:00 AM
@constraint(model, x[2] + x[3] >= 10) # 8:00 AM - 12:00 PM
@constraint(model, x[3] + x[4] >= 7)  # 12:00 PM - 4:00 PM
@constraint(model, x[4] + x[5] >= 12) # 4:00 PM - 8:00 PM
@constraint(model, x[5] + x[6] >= 4)  # 8:00 PM - 12:00 AM

# Solve the model
optimize!(model)

# Print the results
optimal_buses = value.(x)
optimal_total = objective_value(model)

println("Número óptimo de autobuses para cada turno: ", optimal_buses)
println("Número total mínimo de autobuses: ", optimal_total)


Número óptimo de autobuses para cada turno: [4.0, 4.0, 6.0, 8.0, 4.0, 0.0]
Número total mínimo de autobuses: 26.0
